In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import datetime 
from datetime import timedelta ,time, datetime 
import re
from openpyxl import load_workbook
# from msoffcrypto_tool import OfficeFile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
import io
pd.set_option('display.max_columns', None)
pd.set_option('display.notebook_repr_html', True)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 导入数据

In [ ]:
# select t1.all_money, t1.all_postage_money, t2.commodity_premium,t1.status, t1.merchant_name
# from db_digua_business.t_order t1 
# left join db_digua_business.t_order_details t2 on t1.id = t2.order_id 
# where alipay_order_id in (
# SELECT REPLACE(JSON_EXTRACT(JSON_EXTRACT(request_json, "$.bizModel"), "$.orderId"), '"', '') as orderId 
# FROM `db_digua_business`.`t_mini_order_request` 
# WHERE `request_json` LIKE '%alipay.commerce.rent.order.fulfillment.send%' 
# AND `create_time` >= '2025-08-01 00:00:00' 
# AND `create_time` <= '2025-08-24 23:59:59' 
# AND `response_json` LIKE '%Success%')
# # alipay.commerce.rent.order.fulfillment.send 交易组件

# 临时需求

In [2]:
# 发货时间
sql = '''
select t1.all_money, t1.all_postage_money, t2.commodity_premium,t1.status, t1.merchant_name,tolog.go_express_date, tmor.create_time
from db_digua_business.t_order t1 
left join db_digua_business.t_order_details t2 on t1.id = t2.order_id 
left join db_digua_business.t_order_logistics tolog on t1.id = tolog.order_id
left join (
    SELECT REPLACE(JSON_EXTRACT(JSON_EXTRACT(request_json, "$.bizModel"), "$.orderId"), '"', '') as order_id, create_time
    FROM db_digua_business.t_mini_order_request 
    WHERE request_json LIKE '%send%' 
    AND create_time >= '2025-07-26 00:00:00' 
    AND create_time <= '2025-08-24 23:59:59' 
    AND response_json LIKE '%Success%'
) tmor on t1.alipay_order_id = tmor.order_id
where t1.alipay_order_id in (
    SELECT REPLACE(JSON_EXTRACT(JSON_EXTRACT(request_json, "$.bizModel"), "$.orderId"), '"', '') as orderId
    FROM db_digua_business.t_mini_order_request 
    WHERE request_json LIKE '%send%' 
    AND create_time >= '2025-07-26 00:00:00' 
    AND create_time <= '2025-08-24 23:59:59' 
    AND response_json LIKE '%Success%'
)
'''
df_zh = clean.query(sql)
df_zh.shape

(5936, 7)

In [3]:
df = df_zh.copy()
df["create_time"] = pd.to_datetime(df["create_time"], errors='coerce')
df["发货日期"] = df["create_time"].dt.date
df["发货日期"]=pd.to_datetime(df["发货日期"],errors="coerce")
df.loc[:,'总租金'] = df.all_money+ df.all_postage_money + df.commodity_premium
df.tail()

,all_money,all_postage_money,commodity_premium,status,merchant_name,go_express_date,create_time,发货日期,总租金
5931,5978.05,0.0,0.0,4,重庆润禄谦,2025-08-22 12:10:17,2025-08-22 12:10:20,2025-08-22,5978.05
5932,5978.05,0.0,0.0,4,广州欢聚铺,2025-08-23 11:15:47,2025-08-23 11:15:50,2025-08-23,5978.05
5933,5978.05,0.0,0.0,3,海南澜樾店,2025-08-23 11:15:57,2025-08-23 11:16:00,2025-08-23,5978.05
5934,5978.05,0.0,0.0,4,海南澜樾店,2025-08-23 11:15:59,2025-08-23 11:16:02,2025-08-23,5978.05
5935,5978.05,0.0,0.0,3,海南澜樾店,2025-08-24 11:32:13,2025-08-24 11:32:16,2025-08-24,5978.05


In [7]:
# 按发货日期分组，计算总租金
df_group = df.groupby(["发货日期"]).agg({"发货日期": 'count',"总租金": 'sum'}).rename(columns={"发货日期": "订单数"}).reset_index()
# 计算平均总租金
# df_group["平均总租金"] = df_group["总租金"] / df_group["订单数"]
# df_group.loc["汇总", ["订单数", "总租金"]] = df_group[["订单数", "总租金"]].sum()
# df_group.loc["平均", ["订单数", "总租金"]] = df_group[["订单数", "总租金"]].mean()
# df_group.to_excel('F:\需求\七月需求\订单.xlsx', index=False)

# 租后业务量

In [5]:
# ymos.status :状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款
# tprm.rembursement_status :tprm_还款状态, tprm.overdue_type : 逾期类型,M1-M6
# 
sql = '''-- 租后分期应收实收业务量数据统计  
SELECT  
-- ymos.order_id, ymos.money, ymos.part_payment, ymos.refund_date, ymos.reality_refund_date, ymos.sort, ymos.status 分期租金支付状态
-- ,ymos.sesame_promo_money_pay, ymos.promo_money,

-- tprm.purchase_amount, tprm.all_deposit 总押金, tprm.paid_periods 已还期数, tprm.overdue_periods 逾期期数, tprm.rembursement_status tprm_还款状态,tprm.model_number,
-- tprm.now_overdue_days, tprm.commodity_premium 商品溢价费, tprm.overdue_fine 滞纳金, tprm.overdue_type 逾期类型, tprm.specification,tprm.max_overdue_days
-- ,tprm.all_rental
-- , tprm.overdue_periods 逾期期数,

om.status , om.create_time 订单创建时间, om.order_number, om.has_actual
-- , om.order_type, om.order_method
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  status2,
om.end_date as 订单结束时间, om.relet_status, om.overdue,

tmu.true_name ,tmu.id_card_num,tmu.mobile,

-- tod.product_name,
tod.new_actual_money, tod.sku_attributes,

cc.name as channel_name_cc,        -- 来源渠道
cc.channel_type_id,              -- 渠道id
pa.name as activity_name,        -- 活动名称
pa.type

,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
, ym.name as merchant_name
-- ,tolog.go_express_date
from db_digua_business.t_order om
-- db_rent.ya_merchant_order_stages ymos

left join db_digua_business.t_postlease_receivables_monitoring  tprm on om.id = tprm.order_id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
-- left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id

-- 2025.4.29
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
where DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2022-06-01'
-- and om.status = 4
-- and om.overdue = 0
;
'''

df_zhys = clean.query(sql)
# (27787, 62)
df_zhys.shape

(4387686, 21)

In [58]:
df_zh.head()
# 自营订单
# 应收未收金额统计逻辑，订单未到期之前，逾期N期，则为逾期的N期对应的租金；订单到期后，为逾期N期的租金和买断尾款
# 续租、租送没有买断尾款，即为0 new_actual_money-all_rental
# 订单到期时间，om.end_date 与当前时间比较，小于当前时间，则未到期
# 判断逾期订单， 最后将逾期订单的逾期金额累加到逾期金额中
# 续租分期记录只存在于ya_merchant_order_stages
# 资方分类：# name 维客自营账户
#海南鸿丰商业保理有限公司
#杭州金锤网络信息服务有限公司
#成都越蓉金科科技有限公司

# 22,23年订单，如果订单satatus=8，应收未收为0，逾期天数为0，逾期金额也为0

,all_money,all_postage_money,commodity_premium,status,merchant_name
0,100.0,0.0,0.0,8,线下小店
1,100.0,0.0,0.0,8,线下小店
2,100.0,0.0,0.0,8,线下小店
3,100.0,0.0,0.0,8,线下小店
4,100.0,0.0,0.0,8,线下小店


In [ ]:
sql = """select om.id as order_id,om.order_number,om.create_time,tprm.purchase_amount,tod.new_actual_money,tprm.model_number,om.merchant_name,ym.name,om.status
        from db_digua_business.t_order om 
        left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id = om.id
        left join db_digua_business.t_order_details tod on tod.order_id=om.id 
        left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
        where om.status!=1 and om.status!=13"""

# 剔除商家数据

In [6]:
def drop_sj(df_zhys,col1):#,col2
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="星晟数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="蘑菇时间"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="云启德曜"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="人人享租"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="兴鑫兴通讯"].index,inplace=True)  #二手订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="崇胜数码"].index,inplace=True)  #二手订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵新租机"].index,inplace=True)  #拒量订单

    # df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

# df_zhys = drop_sj(df_zhys,'merchant_name')#,'model_number'
# df_zhys.shape

df_zh = drop_sj(df_zhys,'merchant_name')#,'model_number'
df_zh.shape

(4387686, 21)

# 处理日期和备注信息

In [8]:
df = df_zhys.copy()
df["下单日期"] = df["订单创建时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["订单创建时间"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
print(df['年份'].unique())

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["是否进件"] = np.where(df["进件"] == '进件', 1, 0)
df["是否出库"] = np.where(df["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
# df.loc[:,"来源渠道"]=df["channel_name_cc"].fillna("未知渠道")
# df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")

# df['归属渠道'] = np.where(df.归属渠道 == '顶部搜索框的搜索结果页', '搜索渠道', df.归属渠道)
# # conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
# conditions_1 = ['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']
# # 租机宝 、 富士康租机 、 兔丫租赁 、 邦客惠乘车码  -- 不算S量
# # df['归属渠道'] = np.where(df.归属渠道.isin(conditions_1), 'S量', df.归属渠道)
df['是否免人审'] = np.where((df.is_vip=='1')&(df.status_result=='0'), 1 , 0)
df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
# df.loc[:, '分期金额'] = df.groupby('order_id')['money'].transform('max') 
# df.loc[:, '分期金额'] = df[df['sort'] == 5]['money']
# df['是否号卡'] = np.where(df.type == 4, 1, 0)

# df["内存"]=df["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# df["颜色"]=df["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]

[2022 2023 2024 2025]


# 去重

In [10]:
# df = df[df.是否出库==1]
# 先根据order_id、sort去重
# duplicates = df[df.duplicated(subset=['order_id', 'sort'], keep=False)]
# if not duplicates.empty:
#     print("发现重复数据，数量：", len(duplicates))
#     # 可以选择保留最新的记录（根据实际业务逻辑调整）
#     # 这里假设reality_refund_date不为空的是更新的记录
#     df = df.sort_values(['order_id', 'sort', 'refund_date'], ascending=[True, True, False])
#     # 去重，保留每个(order_id, sort)的第一条记录（即最新的）
#     df = df.drop_duplicates(subset=['order_id', 'sort'], keep='first')
    

# 订单去重
dict_status_code = {
            "订单取消": 1,
            "待支付": 2,
            "已退款": 3,
            "待审核": 4,
            "待发货": 5,
            "待收货": 6,
            "租赁中": 7,
            "已完成": 8
        }
df["状态编码"] = df["status2"].map(dict_status_code)
df.sort_values(by=["下单日期", "状态编码"], inplace=True)
# 删除重复单号,需要分期数据时不去重order_id
df.drop_duplicates(subset=["order_number"], inplace=True)
# 删除身份证空值行
df.dropna(subset=["id_card_num"], axis=0, inplace=True)
# 去刷单订单
# df.drop(df[df['total_describes'].str.contains(pat='panli', regex=False) == True].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
# df.drop_duplicates(subset=["order_id"], inplace=True)
# df.drop_duplicates(subset=["order_id"], inplace=True)
# df.drop_duplicates(subset=["true_name", "mobile", "id_card_num", "下单日期","sort"], keep="last", inplace=True)
# df.drop(df[df['true_name'].isin(
#         ["刘鹏", "谢仕程", "潘立", "洪柳", "陈锦奇", "周杰", "卢腾标", "孔靖", "黄娟", "钟福荣", "邱锐杰", "唐林华"
#         , "邓媛斤", "黄子南", "刘莎莎", "赖瑞彤", "孙子文", '淦文豪', '杨明豪', '闫宇龙'])].index, inplace=True)
df.shape

(4306454, 32)

In [11]:
df.groupby('年份').agg({"是否进件":"sum", "是否出库":"sum"})

,是否进件,是否出库
年份,,
2022,7187,845
2023,82157,9687
2024,170044,22906
2025,460113,52063


In [9]:
df_yq = pd.read_excel('F:\需求\倩姐需求\order_statu_202509.xlsx')
df_merge = df.merge(df_yq, how='left', on='order_number')
df_merge.shape# 67664

(908319, 43)

In [10]:
# 自营订单
# 应收未收金额统计逻辑，订单未到期之前，逾期N期，则为逾期的N期对应的租金；订单到期后，为逾期N期的租金和买断尾款
# 续租、租送没有买断尾款，即为0 new_actual_money-all_rental
# 订单到期时间，om.end_date 与当前时间比较，小于当前时间，则未到期
# 判断逾期订单， 最后将逾期订单的逾期金额累加到逾期金额中
# 续租分期记录只存在于ya_merchant_order_stages
# 资方分类：# name 维客自营账户
#海南鸿丰商业保理有限公司
#杭州金锤网络信息服务有限公司
#成都越蓉金科科技有限公司
df_merge.loc[:, '是否到期'] = np.where(df_merge['订单结束时间']<datetime.now(), 1, 0)
df_merge.loc[:, "买断尾款"] = np.where(((df_merge['租赁方案'] == '租完即送')|(df_merge['relet_status']==2)), 0, df_merge['new_actual_money']-df_merge['all_rental'])
df_merge.loc[:, "是否逾期"] = np.where((df_merge['overdue_days_mob']>0), 1, 0)#&(df_merge['overdue']==1)
df_merge['是否逾期'] = np.where((df_merge['年份'].isin([2022, 2023]))&(df_merge['status']==8), 0, df_merge['是否逾期'])
# # 将reality_refund_date改为日期年月日格式
# df_merge["实还日期"] = df_merge["reality_refund_date"].dt.date

# df_merge.loc[:, "实还日期"] = pd.to_datetime(df_merge['应还日期'],errors="coerce")
# df_merge.loc[:, "最小逾期应还日期"] = np.where(df_merge.实还日期.isna(), df_merge.应还日期, df_merge.实还日期)
# # 取出逾期订单最小的应还期数
# df_merge.loc[:, '最小应还期数'] = np.where(df_merge.实还日期.isna(), df_merge.)

# df_merge.loc[:, "租金是否逾期"] = np.where(df_merge.应还日期.isna(), np.where(df_merge.refund_date<datetime.now(), 1, 0), 
#                                     np.where(df_merge.应还日期>df_merge.refund_date, 1, 0))
# df_merge.loc[:, "逾期金额"] = df_merge[df_merge['租金是否逾期'] == 1].groupby('order_id')['money'].transform('sum').fillna(0)

df_merge.loc[:, "逾期金额"] = np.where(df_merge['是否逾期'] == 1, df_merge['分期金额']*df_merge['逾期期数'], 0)
# df_merge['是否逾期'] = np.where((df_merge['年份'].isin(['2022', '2023']))&(df_merge['status']==8), 0, df_merge['是否逾期'])
df_merge.loc[:, "应收未收"] = np.where(df_merge['是否到期'] == 0, df_merge['逾期金额'], df_merge['买断尾款']+df_merge['逾期金额'])
df_merge.loc[:, "资方"] = np.where(df_merge['merchant_name']=='维客自营账户', '维客自营账户', np.where(df_merge['merchant_name']=='海南鸿丰商业保理有限公司', '海南鸿丰商业保理有限公司','其他'))
df_merge.drop_duplicates(subset=["id_card_num", "下单日期"], keep="last", inplace=True)
# # 22,23年订单，如果订单satatus=8，应收未收为0，逾期天数为0，逾期金额也为0,是否逾期为0
# df_merge['']
# print(df_merge['应还日期'])
df_merge.shape

(68398, 49)

In [19]:
# df_merge[['order_number', '资方']].to_excel('F:\需求/倩姐需求/自营逾期订单号.xlsx', index=False)

# 逾期订单数、应收未收金额统计

## 维客自营账户

In [29]:
# df123 = df_wk[df_wk['下单月份']=='2022-10'][['order_number','refund_date', 'reality_refund_date', 'overdue','逾期金额', '应收未收','买断尾款','订单结束时间','是否逾期','overdue_days_mob']]
# # df123[df123['逾期金额']>0]
# df123
# df_merge[(df_merge['资方'] == '维客自营账户')&(df_merge['是否逾期'] == 1)]
# df_wk[(df_wk['年份']==2023)].status.value_counts().sort_values(ascending=False)

status
8     4299
4     1891
5     1366
15       6
Name: count, dtype: int64

In [28]:

# df_merge[df_merge['order_number']=='A20240704014104126']
# df_merge.order_number.value_counts().sort_values(ascending=False)
# df_wk[(df_wk['年份']==2023)].是否逾期.value_counts().sort_values(ascending=False)

是否逾期
0    4365
1    3197
Name: count, dtype: int64

In [32]:
# df_wk[(df_wk['下单月份']=='2023-11')&(df_wk['是否逾期']==1)][['order_number', 'money', '逾期金额','逾期期数','分期金额','merchant_name','status']]

,order_number,money,逾期金额,逾期期数,分期金额,merchant_name,status
108483,A20231101005524111,759.96,5319.72,7.0,759.96,维客自营账户,4
108495,A2023110107482670,300.61,901.83,3.0,300.61,维客自营账户,4
108507,A2023110108403958,395.86,2375.16,6.0,395.86,维客自营账户,4
108519,A2023110110032411,440.74,3525.92,8.0,440.74,维客自营账户,4
108531,A2023110110264054,668.01,6680.10,10.0,668.01,维客自营账户,4
...,...,...,...,...,...,...,...
133005,A2023113012090320,35.60,4074.48,8.0,509.31,维客自营账户,4
133041,A202311301603214,534.43,5878.73,11.0,534.43,维客自营账户,4
133053,A2023113016304747,266.91,2135.28,8.0,266.91,维客自营账户,4
133089,A2023113017141519,467.57,3272.99,7.0,467.57,维客自营账户,4


In [29]:
df_wk = df_merge[(df_merge['资方'] == '维客自营账户')&(df_merge['是否逾期'] == 1)]
df_wk_g = df_wk.groupby(['下单月份']).agg({'是否逾期': 'sum', '应收未收': 'sum'}).rename(columns={'是否逾期': '逾期订单数', '应收未收': '应收未收金额'})
df_wk_g.loc[:, '订单平均逾期金额'] = df_wk_g['应收未收金额'] / df_wk_g['逾期订单数']

df_wk_g.loc['汇总', :] = df_wk_g.sum()
df_wk_g 

,逾期订单数,应收未收金额,订单平均逾期金额
下单月份,,,
2022-07,7.0,53970.48,7710.068571
2022-08,29.0,202900.55,6996.570690
2022-09,20.0,116625.23,5831.261500
2022-10,12.0,67675.55,5639.629167
2022-11,13.0,87819.00,6755.307692
2022-12,3.0,19193.20,6397.733333
2023-01,80.0,470290.73,5878.634125
2023-02,60.0,419712.07,6995.201167
2023-03,30.0,230774.12,7692.470667


In [30]:
df_order_num = df_wk[(df_wk['下单月份']=='2023-11')&(df_wk['是否逾期']==1)][['order_number','all_rental', '应收未收', '逾期期数','分期金额','逾期金额','买断尾款']].rename(columns={'order_number': '订单号', 'all_rental': '应收租金', '应收未收': '应收成本'})
df_order_num.loc['汇总',:] = df_order_num.select_dtypes(include=['number']).sum()
df_order_num#[:20]#.to_excel('F:\需求/倩姐需求/2023年11月逾期订单号.xlsx', index=False)

,订单号,应收租金,应收成本,逾期期数,分期金额,逾期金额,买断尾款
108483,A20231101005524111,9097.30,5319.72,7.0,759.96,5319.72,0.00
108495,A2023110107482670,3598.52,5100.81,3.0,300.61,901.83,4198.98
108507,A2023110108403958,4345.36,5827.30,6.0,395.86,2375.16,3452.14
108519,A2023110110032411,5275.57,7347.65,8.0,440.74,3525.92,3821.73
108531,A2023110110264054,7997.20,11679.60,10.0,668.01,6680.10,4999.50
...,...,...,...,...,...,...,...
133041,A202311301603214,6397.54,9878.29,11.0,534.43,5878.73,3999.56
133053,A2023113016304747,3195.00,4138.18,8.0,266.91,2135.28,2002.90
133089,A2023113017141519,5597.53,6772.76,7.0,467.57,3272.99,3499.77
133514,A2023113008515026,2238.60,1399.50,0.0,186.99,0.00,1399.50


## 海南鸿丰商业保理有限公司

In [142]:
df_hf[(df_hf['下单月份']=='2024-12')&df_hf['是否逾期']==1][['order_id', 'money', '逾期金额','逾期期数','分期金额']]

,order_id,money,逾期金额,逾期期数,分期金额
27490,1815388,0.0,2108.48,4.0,527.12
28137,1846019,1.0,5369.40,10.0,536.94
28256,1851285,1.0,0.00,0.0,536.94
28266,1851946,0.0,527.12,1.0,527.12
28616,1868360,0.0,2635.60,5.0,527.12
...,...,...,...,...,...
29173,1893670,1.0,4308.24,6.0,718.04
29179,1894077,1.0,0.00,0.0,646.08
29181,1894131,1.0,1436.08,2.0,718.04
29182,1894206,1.0,4809.12,8.0,601.14


In [34]:
df_hf = df_merge[(df_merge['资方'] == '海南鸿丰商业保理有限公司')&(df_merge['是否逾期']==1)]
df_hf_g = df_hf.groupby(['下单月份']).agg({'是否逾期': 'sum', '应收未收': 'sum'}).rename(columns={'是否逾期': '逾期订单数', '应收未收': '应收未收金额'})
df_hf_g.loc[:, '订单平均逾期金额'] = df_hf_g['应收未收金额'] / df_hf_g['逾期订单数']
df_hf_g.loc['汇总', :] = df_hf_g.sum()
df_hf_g 

,逾期订单数,应收未收金额,订单平均逾期金额
下单月份,,,
2024-12,89.0,274063.73,3079.367753
2025-01,21.0,68119.08,3243.765714
2025-02,26.0,50568.53,1944.943462
2025-03,72.0,133302.74,1851.426944
2025-04,81.0,128161.73,1582.243580
2025-05,85.0,167597.98,1971.740941
2025-06,102.0,91804.23,900.041471
2025-07,108.0,101338.56,938.320000
2025-08,46.0,44539.10,968.241304


## 其他

In [35]:
df_qt = df_merge[(df_merge['资方'] == '其他')&(df_merge['是否逾期']==1)]
df_qt_g = df_qt.groupby(['下单月份']).agg({'是否逾期': 'sum', '应收未收': 'sum'}).rename(columns={'是否逾期': '逾期订单数', '应收未收': '应收未收金额'})
df_qt_g.loc[:, '订单平均逾期金额'] = df_qt_g['应收未收金额'] / df_qt_g['逾期订单数']
df_qt_g.loc['汇总', :] = df_qt_g.sum()
df_qt_g 

,逾期订单数,应收未收金额,订单平均逾期金额
下单月份,,,
2022-12,20.0,151881.00,7594.050000
2023-01,7.0,41265.45,5895.064286
2023-02,53.0,342072.26,6454.193585
2023-03,83.0,506211.75,6098.936747
2023-04,20.0,184283.88,9214.194000
2023-05,30.0,178274.42,5942.480667
2023-06,80.0,568533.52,7106.669000
2023-08,73.0,509944.19,6985.536849
2023-09,14.0,89550.94,6396.495714


In [36]:
with pd.ExcelWriter('F:\需求\倩姐需求\自营逾期业务量_10.21.xlsx') as writer:
    df_wk_g.to_excel(writer, sheet_name='维客自营账户')
    df_hf_g.to_excel(writer, sheet_name='海南鸿丰商业保理有限公司')
    df_qt_g.to_excel(writer, sheet_name='其他')
    # df_order_num.to_excel(writer, sheet_name='2023年11月逾期订单号', index=False)
    

# 订单业务量数据，按订单号统计

In [87]:
# 计算单个订单业务量数据，按订单id聚合计算总应收租金，总已收租金
df.loc[:, '总应收租金'] = df.groupby('order_id')['money'].transform('sum')
df.loc[:, '总已收租金'] = df[df.分期租金支付状态==3].groupby('order_id')['part_payment'].transform('sum')
df.loc[:, '总未收租金'] = df.loc[:, '总应收租金'] - df.loc[:, '总已收租金']
df.loc[:, '总未收租金'] = np.where(df.loc[:, '总未收租金']<0, 0, df.loc[:, '总未收租金'])
df.loc[:, '剩余总未收'] = df['new_actual_money'] - df['总已收租金']
df.loc[:, '剩余总未收'] = np.where(df.loc[:, '剩余总未收']<0, 0, df.loc[:, '剩余总未收'])
# df.head()
# 按订单id去重，保留每个订单id的第一条记录
df = df.drop_duplicates(subset='order_id', keep='first')
# 订单业务量数据：计算所有订单的总应收租金、总已收租金、总未收租金、剩余总未收、purchase_amount、new_actual_money总和

order_business_summary = {
    '总应收租金(万元)': (df['总应收租金'].sum()/ 10000).round(2),
    '总已收租金(万元)': (df['总已收租金'].sum()/ 10000).round(2),
    '总未收租金(万元)': (df['总未收租金'].sum()/ 10000).round(2),
    '剩余总未收(万元)': (df['剩余总未收'].sum()/ 10000).round(2),
    '采购成本总和(万元)': (df['purchase_amount'].sum()/ 10000).round(2),
    '买断价总和(万元)': (df['new_actual_money'].sum()/ 10000).round(2)
}

# 创建一个DataFrame来存储汇总数据
summary_df = pd.DataFrame(list(order_business_summary.items()), columns=['指标', '金额'])
summary_df
# 买断：59754,8702.4400002
# 采购：39025,3323.15
# 总应收租金：32552,0049.28
# 剩余总应收：41017,3532.89000005
# 剩余总应收租金：17593,4053.65000004


,指标,金额
0,总应收租金(万元),41130.56
1,总已收租金(万元),23462.67
2,总未收租金(万元),17178.20
3,剩余总未收(万元),43039.59
4,采购成本总和(万元),43896.36
5,买断价总和(万元),67387.34


In [83]:
df.shape

(56726, 59)

## 何薪华需求

In [24]:
sql_name = '''
SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
FROM db_digua_business.t_user_verify_order_record tuvor
left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
where tuvor.del_flag = 0 ORDER BY tuvor.update_time
'''
df_name = clean.query(sql_name)

In [25]:
# 拒量的订单2024年12月到2025年6月正常还款的（租赁中）用户没有逾期的订单，需要订单号，用户下单的手机号码，审核人，还款期数（已还）
df_jl = df[(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# 获取订单已还期数，count(part_payment>0)为已还期数
df_jl.loc['已还期数'] = df_jl.groupby('order_id')['part_payment'].transform(lambda x: (x > 0).sum())
df_jl.shape




(1138, 35)

In [29]:

df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# 内连接，获取出库单分配人
df_jl_name = df_jl.merge(df_name_new, on='order_id', how='inner')
df_jl_name_new = df_jl_name[['order_number', 'mobile', '分配人', '已还期数']]
df_jl_name_new.to_excel('F:/需求/薪华需求/拒量非逾期订单.xlsx', index=False)
# df_jl_name_new.shape